In [1]:
import numpy as np
import csv
import pandas as pd
from pathlib import Path
import scipy

In [2]:
# Read the CSV file from the Resources folder into a Pandas DataFrame

df= pd.read_csv(r'C:\Users\dkrenn\Desktop\GW-Bootcamp-Final-Project\Data Analysis\Resources\cleaned_data.csv')
df.head()


,Unnamed: 0,ID,AreaName,VicAge,VicSex,VicRace,StatusCode,Lat,Lon,CrimeType,AgeGroup,AddressType,Arrest
0,0,190326475,Wilshire,999,M,Multiracial,AA,34.0375,-118.3506,Theft,Unknown,Public Space,Yes
1,1,200106753,Central,47,M,Multiracial,IC,34.0444,-118.2628,Theft,40-54,Public Space,No
2,2,200320258,Southwest,19,NaN,,IC,34.0210,-118.3002,Theft,Young Adult,Residential,No
3,3,200907217,Van Nuys,19,M,Multiracial,IC,34.1576,-118.4387,Theft,Young Adult,Buisness,No
4,4,220614831,Hollywood,28,M,Hispanic,IC,34.0944,-118.3277,Theft,25-39,Public Space,No


In [3]:
# Gender and Arrest Cross Tab
cont = pd.crosstab(df["VicSex"], df["Arrest"])
cont

Arrest,No,Yes
VicSex,,
F,291177,32714
M,329897,33363


In [4]:
# Gender and Arrest Chi2
scipy.stats.chi2_contingency(cont)

Chi2ContingencyResult(statistic=165.18503932760916, pvalue=8.334675223426716e-38, dof=1, expected_freq=array([[292745.37755748,  31145.62244252],
       [328328.62244252,  34931.37755748]]))

In [5]:
# Race Recode
race_df=df.loc[df["VicRace"]!=" "]
race =["AANHPI", "AIAN", "Black", "Hispanic", "Multiracial", "White"]
recode = ["Non-white", "Non-white","Non-white","Non-white", "Non-white", "White"]
race_df["RaceRecode"]=race_df["VicRace"].replace(race, recode)
race_df.head()

C:\Users\dkrenn\AppData\Local\Temp\1\ipykernel_18384\2683650618.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  race_df["RaceRecode"]=race_df["VicRace"].replace(race, recode)


,Unnamed: 0,ID,AreaName,VicAge,VicSex,VicRace,StatusCode,Lat,Lon,CrimeType,AgeGroup,AddressType,Arrest,RaceRecode
0,0,190326475,Wilshire,999,M,Multiracial,AA,34.0375,-118.3506,Theft,Unknown,Public Space,Yes,Non-white
1,1,200106753,Central,47,M,Multiracial,IC,34.0444,-118.2628,Theft,40-54,Public Space,No,Non-white
3,3,200907217,Van Nuys,19,M,Multiracial,IC,34.1576,-118.4387,Theft,Young Adult,Buisness,No,Non-white
4,4,220614831,Hollywood,28,M,Hispanic,IC,34.0944,-118.3277,Theft,25-39,Public Space,No,Non-white
5,5,231808869,Southeast,41,M,Hispanic,IC,33.9467,-118.2463,Theft,40-54,Residential,No,Non-white


In [6]:
# Race and Arrest Cross Tab
cont = pd.crosstab(race_df["RaceRecode"], race_df["Arrest"])
cont

Arrest,No,Yes
RaceRecode,,
Non-white,448588,51420
White,165563,13570


In [7]:
# Race and Arrest Chi2
scipy.stats.chi2_contingency(cont)

Chi2ContingencyResult(statistic=1117.669973241428, pvalue=4.7691895084004055e-245, dof=1, expected_freq=array([[452160.02745822,  47847.97254178],
       [161990.97254178,  17142.02745822]]))

In [11]:
# Set Up Data
regress_df =df
Arrest =["No", "Yes"]
recode = [0, 1]
regress_df["ArrestDC"]=regress_df["Arrest"].replace(Arrest, recode)
regress_df["ArrestDC"]=regress_df["ArrestDC"].astype(str).astype(int)
regress_df=df.loc[df["VicAge"]<999]

# Age Regression
from scipy.stats import linregress
import numpy as np

regress_df=regress_df[["ArrestDC", "VicAge"]]
slope, intc, r, p, std_err = linregress(regress_df['VicAge'], regress_df['ArrestDC'])
model=slope*regress_df["VicAge"]+intc
print(str("r value: ") + str(r.round(2)))
print(str("p value: ")+ str(p.round(2)))
print("regression equation: y="+str(slope.round(2))+"*x+"+str(intc.round(2)))

r value: -0.04
p value: 0.0
regression equation: y=-0.0*x+0.13


In [12]:
# Examine Crime Categories
df["CrimeType"].unique()

array(['Theft', 'Assault', 'Sex Crime', 'Other', 'Weapons', 'Robbery',
       'Fraud', 'Property Destruct', 'Kidnapping', 'Driving Offense',
       'Homicide'], dtype=object)

In [13]:
# Recode Crime Types
crime =['Theft', 'Assault', 'Sex Crime', 'Other', 'Weapons', 'Robbery',
       'Fraud', 'Property Destruct', 'Kidnapping', 'Driving Offense',
       'Homicide']
violent = ["No", "Yes", "Yes", "No", "Yes","Yes","No", "No", "Yes", "No", "Yes"]
df["Violent"]=df["CrimeType"].replace(crime, violent)
df.head()

,Unnamed: 0,ID,AreaName,VicAge,VicSex,VicRace,StatusCode,Lat,Lon,CrimeType,AgeGroup,AddressType,Arrest,ArrestDC,Violent
0,0,190326475,Wilshire,999,M,Multiracial,AA,34.0375,-118.3506,Theft,Unknown,Public Space,Yes,1,No
1,1,200106753,Central,47,M,Multiracial,IC,34.0444,-118.2628,Theft,40-54,Public Space,No,0,No
2,2,200320258,Southwest,19,NaN,,IC,34.0210,-118.3002,Theft,Young Adult,Residential,No,0,No
3,3,200907217,Van Nuys,19,M,Multiracial,IC,34.1576,-118.4387,Theft,Young Adult,Buisness,No,0,No
4,4,220614831,Hollywood,28,M,Hispanic,IC,34.0944,-118.3277,Theft,25-39,Public Space,No,0,No


In [14]:
# Gender and Violence Cross Tab
cont = pd.crosstab(df["VicSex"], df["Violent"])
cont

Violent,No,Yes
VicSex,,
F,185441,138450
M,222713,140547


In [15]:
# Gender and Violent Chi2
scipy.stats.chi2_contingency(cont)

Chi2ContingencyResult(statistic=1167.4768302663992, pvalue=7.137927921721377e-256, dof=1, expected_freq=array([[192384.79928575, 131506.20071425],
       [215769.20071425, 147490.79928575]]))

In [16]:
# Race Recode
race_df=df.loc[df["VicRace"]!=" "]
race =["AANHPI", "AIAN", "Black", "Hispanic", "Multiracial", "White"]
recode = ["Non-white", "Non-white","Non-white","Non-white", "Non-white", "White"]
race_df["RaceRecode"]=race_df["VicRace"].replace(race, recode)
race_df.head()

C:\Users\dkrenn\AppData\Local\Temp\1\ipykernel_18384\2683650618.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  race_df["RaceRecode"]=race_df["VicRace"].replace(race, recode)


,Unnamed: 0,ID,AreaName,VicAge,VicSex,VicRace,StatusCode,Lat,Lon,CrimeType,AgeGroup,AddressType,Arrest,ArrestDC,Violent,RaceRecode
0,0,190326475,Wilshire,999,M,Multiracial,AA,34.0375,-118.3506,Theft,Unknown,Public Space,Yes,1,No,Non-white
1,1,200106753,Central,47,M,Multiracial,IC,34.0444,-118.2628,Theft,40-54,Public Space,No,0,No,Non-white
3,3,200907217,Van Nuys,19,M,Multiracial,IC,34.1576,-118.4387,Theft,Young Adult,Buisness,No,0,No,Non-white
4,4,220614831,Hollywood,28,M,Hispanic,IC,34.0944,-118.3277,Theft,25-39,Public Space,No,0,No,Non-white
5,5,231808869,Southeast,41,M,Hispanic,IC,33.9467,-118.2463,Theft,40-54,Residential,No,0,No,Non-white


In [17]:
# Race and Violent Cross Tab
cont = pd.crosstab(race_df["RaceRecode"], race_df["Violent"])
cont

Violent,No,Yes
RaceRecode,,
Non-white,270672,229336
White,131528,47605


In [18]:
# Race and Violent Chi2
scipy.stats.chi2_contingency(cont)

Chi2ContingencyResult(statistic=20322.933384006938, pvalue=0.0, dof=1, expected_freq=array([[296114.08764896, 203893.91235104],
       [106085.91235104,  73047.08764896]]))

In [19]:
# Set Up Data
regress_df =df
Arrest =["No", "Yes"]
recode = [0, 1]
regress_df["ViolentDC"]=regress_df["Violent"].replace(Arrest, recode)
regress_df["ViolentDC"]=regress_df["ViolentDC"].astype(str).astype(int)
regress_df=df.loc[df["VicAge"]<999]

# Age Regression
from scipy.stats import linregress
import numpy as np

regress_df=regress_df[["ViolentDC", "VicAge"]]
slope, intc, r, p, std_err = linregress(regress_df['VicAge'], regress_df['ViolentDC'])
model=slope*regress_df["VicAge"]+intc
print(str("r value: ") + str(r.round(2)))
print(str("p value: ")+ str(p.round(2)))
print("regression equation: y="+str(slope.round(2))+"*x+"+str(intc.round(2)))

r value: -0.14
p value: 0.0
regression equation: y=-0.0*x+0.58
